In [169]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF
import re
import os
from datetime import datetime, timedelta
os.getcwd()
df = pd.DataFrame()

In [324]:
df = pd.read_csv('./dws_dsc_activity_hourly_detail_di.csv', nrows = 15555)
re1 = re.compile(r'(?<=\.).+')
df.columns = [re1.findall(i)[0] for i in list(df.columns.to_numpy())]


In [293]:
df.columns

Index(['ou_code', 'ou_name', 'bg_code', 'bg_name_cn', 'customer_id',
       'customer_name', 'year', 'quarter', 'month', 'operation_day', 'hour',
       'wms_warehouse_id', 'user_id', 'activity_type', 'activity_sub_type',
       'activity_start_time', 'activity_end_time', 'lpn', 'sku_code',
       'order_id', 'from_location', 'to_location', 'qty', 'create_time',
       'internal_container_num', 'internal_id', 'activity_id',
       'wms_company_id', 'inc_day'],
      dtype='object')

In [325]:
df['activity_end_time'] = pd.to_datetime(df['activity_end_time']).fillna(timedelta(0))
df['activity_start_time'] = pd.to_datetime(df['activity_start_time']).fillna(timedelta(0))

In [326]:
"""
d单个activity的耗费时间.
"""
df['time_gap_inner'] = df['activity_end_time']-df['activity_start_time']
df['time_gap_inner'] = df['time_gap_inner'].fillna(
    timedelta(0)
    ).apply(timedelta.total_seconds)
df['time_gap_inner'] = [0 if i< 0 else i for i in df['time_gap_inner']]

"""
前一个activity到后一个activity相差的时间\.
"""
df = df.sort_values('activity_start_time')
df['time_gap_outer'] = df\
    .groupby(['user_id','inc_day'], dropna = False)['activity_start_time'].transform('diff')
df['time_gap_outer'] = df['time_gap_outer'].fillna(timedelta(0)).apply(timedelta.total_seconds)
df['time_gap_outer'] = [0 if i< 0 else i for i in df['time_gap_outer']]

"""
time gap of the day.
"""
mid1 = df.groupby(['inc_day', 'user_id']).agg(
    a = ('activity_start_time','min'), 
    b  = ('activity_start_time', 'max'))\
        [['a','b']].diff(axis = 1)['b'].apply(timedelta.total_seconds).reset_index()
mid1.columns = ['inc_day','user_id','time_gap_today']
df = df.merge(mid1, on = ['inc_day', 'user_id'], how = 'left')


In [327]:
def get_hour():
    list = ['activity_start_time', 'activity_end_time','create_time']
    
    for i in list:
        listnew = i + '_hour'
        df[listnew] = df[i].astype(str).str.slice(12,)
    return df
df = get_hour()
df[['activity_start_time_hour','activity_start_time']]

In [328]:
# df.groupby(['inc_day', 'user_id']).agg(
#     a = ('activity_start_time','min'), 
#     b  = ('activity_start_time', 'max'))[['a','b']].diff(axis = 1)['b']\
# .apply(timedelta.total_seconds).reset_index()
# df[df['user_id'] == 'DSCSZXP0325'][[
# 'activity_start_time','time_gap_inner','time_gap_outer','time_gap_today'
# ]].sort_values('activity_start_time')

In [302]:
df[['ou_code', 'ou_name', 'bg_code', 'bg_name_cn', 'customer_id',
       'customer_name', 'hour',
       'wms_warehouse_id', 'user_id', 'activity_type', 'activity_sub_type',
       'activity_start_time', 'activity_end_time', 'lpn', 'sku_code',
       'order_id', 'from_location', 'to_location', 'qty', 'create_time',
       'activity_id', 'inc_day',
       'time_gap_inner','time_gap_outer','time_gap_today']].\
              to_csv('./activi_for.csv', index=None, encoding='utf_8_sig')